Create testing data for 10 evaluation frequencies

In [71]:
from pathlib import Path
import numpy as np
import pandas as pd
import pandas.testing as pdt
from resistics.testing import time_metadata_mt
from resistics.decimate import get_eval_freqs_size, DecimationSetup
from resistics.spectra import SpectraLevelMetadata, SpectraMetadata, SpectraData
from resistics.gather import QuickGather
from resistics.transfunc import ImpedanceTensor
from resistics.regression import RegressionPreparerGathered, SolverScikitOLS

np.random.seed(42)

Need to generate:
- Values for the components of the impedance tensor for a given number of evaluation frequencies
- Values for the regressors, Hx and Hy
Using this, can calculate values for the observations Ex and Ey

In [72]:
def rand_complex(low: int, high: int) -> complex:
    """Generate a random complex number that is integer"""
    return complex(np.random.randint(low, high), np.random.randint(low, high))


# evaluation frequencies
fs = 256
n_evals = 10
n_levels = 2
n_wins = 50
if n_evals % n_levels != 0:
    raise ValueError(f"{n_evals=} not divisible by {n_levels=}")
per_level = n_evals // n_levels
eval_freqs = get_eval_freqs_size(fs, n_evals)



Calculate the decimation paramters

In [73]:
dec_setup = DecimationSetup(
    n_levels=n_levels, per_level=per_level, eval_freqs=eval_freqs.tolist()
)
dec_params = dec_setup.run(fs)
dec_params.summary()


{
    'fs': 256.0,
    'n_levels': 2,
    'per_level': 5,
    'min_samples': 256,
    'eval_freqs': [
        64.0,
        45.25483399593904,
        32.0,
        22.62741699796952,
        16.0,
        11.31370849898476,
        8.0,
        5.65685424949238,
        4.0,
        2.82842712474619
    ],
    'dec_factors': [1, 4],
    'dec_increments': [1, 4],
    'dec_fs': [256.0, 64.0]
}


In [74]:
levels_fs = dec_params.dec_fs
eval_freqs_for_levels = {}
for ilevel in range(n_levels):
    eval_freqs_for_levels[ilevel] = dec_params.get_eval_freqs(ilevel)

for ilevel, freqs in eval_freqs_for_levels.items():
    print(ilevel, freqs)

0 [64.0, 45.25483399593904, 32.0, 22.62741699796952, 16.0]
1 [11.31370849898476, 8.0, 5.65685424949238, 4.0, 2.82842712474619]


Generate the expected outputs

In [75]:
# tensor components
components = ["ExHx", "ExHy", "EyHx", "EyHy"]
temp = []
# calculate some values
for ifreq in range(n_evals):
    f_c = {comp: rand_complex(-10, 10) for comp in components}
    temp.append(f_c)
tensor = pd.DataFrame.from_records(temp)
tensor.index = eval_freqs
print(tensor)


                ExHx       ExHy      EyHx       EyHy
64.000000  -4.0+9.0j   4.0+0.0j -3.0-4.0j   8.0+0.0j
45.254834   0.0-7.0j  -3.0-8.0j -9.0+1.0j  -5.0-9.0j
32.000000 -10.0+1.0j   1.0+6.0j -1.0+5.0j   4.0+4.0j
22.627417   8.0+1.0j   9.0-8.0j -6.0+8.0j  -4.0-2.0j
16.000000  -4.0+7.0j  -7.0+3.0j  7.0-2.0j  -9.0+9.0j
11.313708   4.0-4.0j   1.0-3.0j  4.0-8.0j   3.0+6.0j
8.000000   -7.0+7.0j  -3.0-7.0j -9.0-5.0j  -1.0-7.0j
5.656854    7.0+1.0j  -9.0-1.0j -7.0+3.0j   5.0+4.0j
4.000000   -3.0+3.0j  -3.0+5.0j  2.0+7.0j   4.0+2.0j
2.828427   -2.0+4.0j  2.0-10.0j -4.0-2.0j -10.0+1.0j


Create the data for the evaluation frequencies

In [76]:
chans = ["Hx", "Hy", "Ex", "Ey"]
n_chans = len(chans)
low = -10
high = 10
data_array = np.empty((n_evals, n_chans, n_wins), dtype=np.complex128)
for ifreq in range(n_evals):
    Hx = np.random.randint(low=low, high=high, size=n_wins)
    Hy = np.random.randint(low=low, high=high, size=n_wins)
    Ex = (
        tensor.loc[eval_freqs[ifreq], "ExHx"] * Hx
        + tensor.loc[eval_freqs[ifreq], "ExHy"] * Hy
    )
    Ey = (
        tensor.loc[eval_freqs[ifreq], "EyHx"] * Hx
        + tensor.loc[eval_freqs[ifreq], "EyHy"] * Hy
    )
    data_array[ifreq] = np.array([Hx, Hy, Ex, Ey])
# transpose so we have the following shape 
# n_wins, n_chans, per_level
data_array = data_array.transpose()
print(data_array.shape)



(50, 4, 10)


Get the data in the right shape. For each decimation level. This should be:

n_wins x n_chans x n_freqs

In [77]:
data = {}
for ilevel in range(n_levels):
    istart = ilevel * per_level
    iend = istart + per_level
    data[ilevel] = data_array[...,istart:iend]
    print(data[ilevel].shape)

(50, 4, 5)
(50, 4, 5)


Now let's put this in evaluation frequency object, which is a SpectraData object

In [78]:
# create the levels metadata
levels_metadata = []
for ilevel, level_fs in enumerate(levels_fs):
    levels_metadata.append(
        SpectraLevelMetadata(
            fs=level_fs,
            n_wins=n_wins,
            win_size=20,
            olap_size=5,
            index_offset=0,
            n_freqs=per_level,
            freqs=eval_freqs_for_levels[ilevel],
        )
    )
    levels_metadata[-1].summary()


{
    'fs': 256.0,
    'n_wins': 50,
    'win_size': 20,
    'olap_size': 5,
    'index_offset': 0,
    'n_freqs': 5,
    'freqs': [64.0, 45.25483399593904, 32.0, 22.62741699796952, 16.0]
}
{
    'fs': 64.0,
    'n_wins': 50,
    'win_size': 20,
    'olap_size': 5,
    'index_offset': 0,
    'n_freqs': 5,
    'freqs': [
        11.31370849898476,
        8.0,
        5.65685424949238,
        4.0,
        2.82842712474619
    ]
}


Make the SpectraMetadata

In [79]:
metadata_dict = time_metadata_mt().dict()
metadata_dict["chans"] = chans
metadata_dict["fs"] = levels_fs
metadata_dict["n_levels"] = len(levels_metadata)
metadata_dict["levels_metadata"] = levels_metadata
metadata_dict["ref_time"] = metadata_dict["first_time"]
spec_metadata = SpectraMetadata(**metadata_dict)
spec_metadata.summary()

{
    'file_info': None,
    'fs': [256.0, 64.0],
    'chans': ['Hx', 'Hy', 'Ex', 'Ey'],
    'n_chans': 4,
    'n_levels': 2,
    'first_time': '2020-01-01 00:00:00.000000_000000_000000_000000',
    'last_time': '2020-01-01 00:00:01.000000_000000_000000_000000',
    'system': '',
    'serial': '',
    'wgs84_latitude': -999.0,
    'wgs84_longitude': -999.0,
    'easting': -999.0,
    'northing': -999.0,
    'elevation': -999.0,
    'chans_metadata': {
        'Ex': {
            'name': 'Ex',
            'data_files': ['Ex.ascii'],
            'chan_type': 'electric',
            'chan_source': None,
            'sensor': '',
            'serial': '',
            'gain1': 1.0,
            'gain2': 1.0,
            'scaling': 1.0,
            'chopper': False,
            'dipole_dist': 1.0,
            'sensor_calibration_file': '',
            'instrument_calibration_file': ''
        },
        'Ey': {
            'name': 'Ey',
            'data_files': ['Ex.ascii'],
            'cha

Make the SpectraData

In [80]:
spec_data = SpectraData(spec_metadata, data)


2023-04-28 14:58:10.104 | DEBUG    | resistics.spectra:__init__:97 - Creating SpectraData with data type complex128


In [81]:
dir_path = Path("check")
tf = ImpedanceTensor()
gathered_data = QuickGather().run(dir_path, dec_params, tf, spec_data)

2023-04-28 14:58:10.128 | INFO     | resistics.gather:run:847 - Quick gathering data for regression prepartion


Now do the regression and check the results

In [82]:
reg_data = RegressionPreparerGathered().run(tf, gathered_data)


2023-04-28 14:58:10.156 | INFO     | resistics.regression:run:388 - Preparing regression data
2023-04-28 14:58:10.157 | INFO     | resistics.regression:run:389 - Out chans site: check
2023-04-28 14:58:10.158 | INFO     | resistics.regression:run:390 - Out chans: ['Ex', 'Ey']
2023-04-28 14:58:10.158 | INFO     | resistics.regression:run:391 - In chans site: check
2023-04-28 14:58:10.159 | INFO     | resistics.regression:run:392 - In chans: ['Hx', 'Hy']
2023-04-28 14:58:10.159 | INFO     | resistics.regression:run:393 - Cross chans site: check
2023-04-28 14:58:10.160 | INFO     | resistics.regression:run:394 - Cross chans: ['Hx', 'Hy']
2023-04-28 14:58:10.160 | INFO     | resistics.regression:_get_regression_data:419 - Preparing regression data for 10 frequencies
100%|██████████| 10/10 [00:00<00:00, 3962.87it/s]


In [85]:
soln = SolverScikitOLS().run(reg_data)
soln.summary()

2023-04-28 14:59:41.666 | INFO     | resistics.regression:_solve:782 - Solving for 10 evaluation frequencies
100%|██████████| 10/10 [00:00<00:00, 1243.79it/s]

{
    'file_info': None,
    'tf': {
        'name': 'ImpedanceTensor',
        'variation': 'default',
        'out_chans': ['Ex', 'Ey'],
        'in_chans': ['Hx', 'Hy'],
        'cross_chans': ['Hx', 'Hy'],
        'n_out': 2,
        'n_in': 2,
        'n_cross': 2
    },
    'freqs': [
        64.0,
        45.25483399593904,
        32.0,
        22.62741699796952,
        16.0,
        11.31370849898476,
        8.0,
        5.65685424949238,
        4.0,
        2.82842712474619
    ],
    'components': {
        'ExHx': {
            'real': [
                -4.0,
                -7.995730771937587e-16,
                -10.000000000000009,
                8.000000000000004,
                -4.000000000000002,
                4.000000000000001,
                -6.999999999999999,
                7.0,
                -3.0000000000000004,
                -2.0000000000000013
            ],
            'imag': [
                9.000000000000002,
                -6.999999999999996

Now check the results with the testing data

In [87]:
soln_dict = {}
for comp in tensor.columns:
    soln_dict[comp] = soln.get_component(comp)
tensor_estimated = pd.DataFrame(data=soln_dict, index=soln.freqs)
print(tensor)
print(tensor_estimated)

                ExHx       ExHy      EyHx       EyHy
64.000000  -4.0+9.0j   4.0+0.0j -3.0-4.0j   8.0+0.0j
45.254834   0.0-7.0j  -3.0-8.0j -9.0+1.0j  -5.0-9.0j
32.000000 -10.0+1.0j   1.0+6.0j -1.0+5.0j   4.0+4.0j
22.627417   8.0+1.0j   9.0-8.0j -6.0+8.0j  -4.0-2.0j
16.000000  -4.0+7.0j  -7.0+3.0j  7.0-2.0j  -9.0+9.0j
11.313708   4.0-4.0j   1.0-3.0j  4.0-8.0j   3.0+6.0j
8.000000   -7.0+7.0j  -3.0-7.0j -9.0-5.0j  -1.0-7.0j
5.656854    7.0+1.0j  -9.0-1.0j -7.0+3.0j   5.0+4.0j
4.000000   -3.0+3.0j  -3.0+5.0j  2.0+7.0j   4.0+2.0j
2.828427   -2.0+4.0j  2.0-10.0j -4.0-2.0j -10.0+1.0j
                ExHx       ExHy      EyHx       EyHy
64.000000  -4.0+9.0j   4.0+0.0j -3.0-4.0j   8.0-0.0j
45.254834  -0.0-7.0j  -3.0-8.0j -9.0+1.0j  -5.0-9.0j
32.000000 -10.0+1.0j   1.0+6.0j -1.0+5.0j   4.0+4.0j
22.627417   8.0+1.0j   9.0-8.0j -6.0+8.0j  -4.0-2.0j
16.000000  -4.0+7.0j  -7.0+3.0j  7.0-2.0j  -9.0+9.0j
11.313708   4.0-4.0j   1.0-3.0j  4.0-8.0j   3.0+6.0j
8.000000   -7.0+7.0j  -3.0-7.0j -9.0-5.0j  -1.